In [ ]:

from google.colab import drive
import pandas as pd
from datetime import datetime
from os import path



drive.mount('/content/drive')

path_data = "/content/drive/MyDrive/Thesis/Data Folder/data.csv"
path_market_data= "/content/drive/MyDrive/Thesis/Data Folder/market_data.csv"
path_code_map = "/content/drive/MyDrive/Thesis/Data Folder/code_map.csv"
path_competitors_data = "/content/drive/MyDrive/Thesis/Data Folder/Antagonismos_Data.csv"


df_data= pd.read_csv(path_data)
df_market_data=pd.read_csv(path_market_data)
df_code_map=pd.read_csv(path_code_map)
df_competitors_data=pd.read_csv(path_competitors_data)


df_data=df_data.drop(labels=['orders_id','customers_id'],axis=1)                              #drop useless columns
df_data=df_data[df_data['price_per_unit'] !=0 ]                                               #drop rows where price equals 0(products are gifts)
df_data=df_data.dropna()                                                                      #drop na rows
df_data.date_purchased=pd.to_datetime(df_data['date_purchased'],format='%Y-%m-%d %H:%M:%S')   #convert  date_purchased to datetime format
df_data.date_purchased=pd.to_datetime(df_data['date_purchased']).dt.date                      # discard hours,minutes and seconds from date_purchased
df_data['date_purchased'] = pd.to_datetime(df_data['date_purchased'])                         #convert to datetime
df_market_data['Date']=pd.to_datetime(df_market_data['Date'])
df_competitors_data['Date']=pd.to_datetime(df_competitors_data['Date'])



#function to match SKU with encrypted item code in data
def match_sku(SKU):
  if SKU in df_code_map.values:
    return df_code_map.loc[df_code_map['itemcode']==SKU].iloc[0][0]
  else:
    print('SKU not founf in data')
    return False

#function to get the competitors price history  of a singel item
def competitors_price(SKU):
 return df_competitors_data[df_competitors_data['SKU']==SKU].resample('W',on='Date').agg({'lianikiMeFpaAfterDiscount':'mean'})


#function to get the market sales   of a singel item
def market_sales(SKU):
  return df_market_data[df_market_data['SKU']==str(SKU)].resample('W',on='Date').agg({'Market Sales (Quantity)':'sum'})

#function to get the sales history of an item
def sales_history(products_id):
    d=df_data[df_data['products_id'] ==products_id].resample('W',on='date_purchased').agg({'products_quantity':'sum','price_per_unit':'mean'})
    d['price_per_unit']=d['price_per_unit'].fillna(0)
    return d

#function to get the complete item data
def item(SKU):
  products_id=match_sku(SKU)
  if products_id!=False:
    df1=sales_history(products_id)
    df2=competitors_price(SKU)
    df3=market_sales(SKU)
    df=pd.concat([df1, df2,df3], axis=1)
    df=df[df['products_quantity'].notna()]
    df=fix_data(df)
  return df


#fix 0 sales dates.1 for mape prediction when >5% and mean >5%
def fix_data(df):
  count=(df['products_quantity']==0).sum()
  if count/len(df)*100>5:
    df['products_quantity'].replace(to_replace = 0, value = 1, inplace=True)
  else:
    df['products_quantity'].replace(to_replace = 0, value =df['products_quantity'].mean(), inplace=True)
    df['price_per_unit'].replace(to_replace = 0, value =df.price_per_unit.mean(), inplace=True)
  return df

In [4]:

from google.colab import drive
import pandas as pd
from datetime import datetime
from os import path



drive.mount('/content/drive')

path_data = "/content/drive/MyDrive/Thesis/Data Folder/data.csv"
path_market_data= "/content/drive/MyDrive/Thesis/Data Folder/market_data.csv"
path_code_map = "/content/drive/MyDrive/Thesis/Data Folder/code_map.csv"
path_competitors_data = "/content/drive/MyDrive/Thesis/Data Folder/Antagonismos_Data.csv"


df_data= pd.read_csv(path_data)
df_market_data=pd.read_csv(path_market_data)
df_code_map=pd.read_csv(path_code_map)
df_competitors_data=pd.read_csv(path_competitors_data)

Mounted at /content/drive


In [13]:
df_data

,orders_id,customers_id,products_quantity,products_id,price_per_unit,date_purchased
0,c214dd2c8d3a8282e5d297fe2babb60f,ec2dd97b3142e7a0d30517eb57998209,15,e8c9d6ea2dc1dc231d3efe83da289566,0.00000,1964-07-03 18:29:47
1,8567430b01acb190bf83c3664ebc2ede,ad4bc386e1a73a372ce5caeaf6857294,2,e8c9d6ea2dc1dc231d3efe83da289566,2.45091,1964-08-15 18:02:51
2,9682b87e41baecb81d4dcd44d6a20dc6,864ea7e3e778916178564bd8e85a4d79,5,e8c9d6ea2dc1dc231d3efe83da289566,2.45091,1964-09-09 09:33:50
3,d0a7aa8948a8a61ee28819c1831d4b8f,d0ed65e1a5d87353fee0795f2784f5fc,1,e8c9d6ea2dc1dc231d3efe83da289566,2.43285,1965-04-07 23:17:56
4,12a6616c1184cd586f978a09bde635c9,a4e9dcb59de9652095f9ab4e452f4a0e,1,76a8ce65a17f2adf589ccd45d1041eab,0.00000,1964-01-10 12:08:58
...,...,...,...,...,...,...
3090884,d2edb8f8dea68847fdd3c2297d9ea961,7325b7a5abf65757bc46f3b814187a6a,1,a03ea55422acb86c8a78abb2d53fca12,24.29700,1969-12-10 13:14:02
3090885,645ca514871a546afbba1d031c89ed97,5ce0099a9ffb9f6b1a6785cf9608ddcc,1,a03ea55422acb86c8a78abb2d53fca12,28.56000,1970-02-25 20:08:12
3090886,7e4e81885a1f51839e56be0b20c47669,feacf4c040b6b8cb184aa7d0f7acbc59,1,a03ea55422acb86c8a78abb2d53fca12,28.56000,1970-03-31 17:22:06
3090887,3aeddd541ccc04295958298523413543,bcae7b3a6d645e4f27930cca32e85f6b,1,a03ea55422acb86c8a78abb2d53fca12,28.56000,1970-05-11 14:31:24
